In [24]:
import ee
import numpy as np
import pandas as pd
import datetime
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction import DictVectorizer

In [2]:
ee.Authenticate()

True

In [3]:
# Initialize Earth Engine
ee.Initialize()

In [33]:
# Define the Corn Belt states
states = ["Iowa", "Illinois", "Indiana", "Minnesota", "Wisconsin"]

# Define the date range for the full month of August 2022
dates = pd.date_range(start="2022-05-07", end="2022-08-31", freq='W')

# Helper function to find the closest available Sentinel-2 image
def get_closest_sentinel_image(county_geom, date):
    """Find the closest available Sentinel-2 image."""
    start_date = (pd.to_datetime(date) - pd.Timedelta(days=3)).strftime('%Y-%m-%d')
    end_date = (pd.to_datetime(date) + pd.Timedelta(days=3)).strftime('%Y-%m-%d')

    sentinel = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate(start_date, end_date) \
        .filterBounds(county_geom) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
        .sort('system:time_start') \
        .first()

    if sentinel.getInfo() is None:
        print(f"No Sentinel-2 image available between {start_date} and {end_date}.")
        return None

    band_names = sentinel.bandNames().getInfo()
    required_bands = {'B2', 'B3', 'B4', 'B8', 'B11'}
    if not required_bands.issubset(band_names):
        print(f"Sentinel-2 image is missing one or more required bands.")
        return None

    return sentinel

# Helper function to calculate relevant satellite indices
def calculate_indices(sentinel):
    """Calculate NDVI, EVI, GNDVI, NDWI, and SAVI from Sentinel-2 data."""
    ndvi = sentinel.normalizedDifference(['B8', 'B4']).rename('NDVI')
    evi = sentinel.expression(
        '2.5 * ((B8 - B4) / (B8 + 6 * B4 - 7.5 * B2 + 1))',
        {'B8': sentinel.select('B8'), 'B4': sentinel.select('B4'), 'B2': sentinel.select('B2')}
    ).rename('EVI')
    gndvi = sentinel.normalizedDifference(['B8', 'B3']).rename('GNDVI')
    ndwi = sentinel.normalizedDifference(['B8', 'B11']).rename('NDWI')
    savi = sentinel.expression(
        '((B8 - B4) / (B8 + B4 + 0.5)) * 1.5',
        {'B8': sentinel.select('B8'), 'B4': sentinel.select('B4')}
    ).rename('SAVI')

    return ndvi.addBands([evi, gndvi, ndwi, savi])

# Helper function to get fields with indices and crop type for a county on a given date
def get_fields_for_county(county_geom, date):
    """Retrieve fields with indices and crop type for a given county and date."""
    # Load the Cropland Data Layer (CDL) for crop types
    cdl = ee.ImageCollection("USDA/NASS/CDL") \
        .filterDate('2022-01-01', '2022-12-31').first() \
        .select(['cropland']).clip(county_geom)

    # Get the closest available Sentinel-2 image
    sentinel = get_closest_sentinel_image(county_geom, date)

    if sentinel is None:
        print(f"No valid Sentinel-2 data available for this county on {date}.")
        return pd.DataFrame()  # Return an empty DataFrame if no data available

    # Calculate indices
    indices = calculate_indices(sentinel)

    # Mask indices with the CDL to focus on cropland areas
    masked_indices = indices.updateMask(cdl.gt(0))

    # Convert the CDL to polygons (fields) and assign unique field IDs
    fields = cdl.reduceToVectors(
        geometry=county_geom,
        scale=300,
        geometryType='polygon',
        maxPixels=1e10
    ).map(lambda f: f.set('field_id', f.id()))  # Assign field ID

    # Compute field-level statistics for each index
    def compute_field_stats(field):
        stats = masked_indices.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=field.geometry(),
            scale=30,
            maxPixels=1e10,
            bestEffort=True
        )
        return field.set(stats)

    # Map the statistics function over the fields
    field_data = fields.map(compute_field_stats)

    # Filter fields with valid NDVI
    valid_fields = field_data.filter(ee.Filter.notNull(['NDVI']))

    # Randomly sample 1,000 valid fields
    sampled_fields = valid_fields.randomColumn('random').sort('random').limit(100)

    # Convert the result to a Pandas DataFrame
    def ee_to_pandas(fc):
        """Convert an EE FeatureCollection to a Pandas DataFrame."""
        features = fc.getInfo()['features']
        data = [{**f['properties'], 'geometry': f['geometry']} for f in features]
        return pd.DataFrame(data)

    return ee_to_pandas(sampled_fields)

# Initialize an empty DataFrame to store all results
all_data = pd.DataFrame()

# Loop over states and their counties
for state in states:
    print(f"Processing state: {state}")
    # Load the counties for the given state
    counties = ee.FeatureCollection("TIGER/2018/Counties") \
        .filter(ee.Filter.eq('STATEFP', ee.FeatureCollection("TIGER/2018/States")
                             .filter(ee.Filter.eq('NAME', state)).first().get('STATEFP')))

    # Loop over each county and each date in August
    for date in dates:
        for county in counties.toList(counties.size()).getInfo():
            county_geom = ee.Geometry(ee.Feature(county).geometry())

            print(f"Processing {county['properties']['NAME']} on {date.date()}...")
            # Get the fields for this county on this date
            df = get_fields_for_county(county_geom, date.strftime('%Y-%m-%d'))

            if not df.empty:
                df['state'] = state
                df['county'] = county['properties']['NAME']
                df['date'] = date
                # Append the data to the main DataFrame
                all_data = pd.concat([all_data, df], ignore_index=True)

Processing state: Iowa
Processing Plymouth on 2022-05-08...
Processing Woodbury on 2022-05-08...
Processing Carroll on 2022-05-08...
No Sentinel-2 image available between 2022-05-05 and 2022-05-11.
No valid Sentinel-2 data available for this county on 2022-05-08.
Processing Dubuque on 2022-05-08...
No Sentinel-2 image available between 2022-05-05 and 2022-05-11.
No valid Sentinel-2 data available for this county on 2022-05-08.
Processing Jefferson on 2022-05-08...
No Sentinel-2 image available between 2022-05-05 and 2022-05-11.
No valid Sentinel-2 data available for this county on 2022-05-08.
Processing Webster on 2022-05-08...
No Sentinel-2 image available between 2022-05-05 and 2022-05-11.
No valid Sentinel-2 data available for this county on 2022-05-08.
Processing Lee on 2022-05-08...
No Sentinel-2 image available between 2022-05-05 and 2022-05-11.
No valid Sentinel-2 data available for this county on 2022-05-08.
Processing Marshall on 2022-05-08...
No Sentinel-2 image available bet

KeyboardInterrupt: 

In [34]:
all_data

,EVI,GNDVI,NDVI,NDWI,SAVI,count,field_id,label,random,geometry,state,county,date
0,0.628393,0.404195,0.298536,-0.163510,0.447747,11,+7735+3224,176,0.000091,"{'geodesic': False, 'type': 'Polygon', 'coordi...",Iowa,Plymouth,2022-05-08
1,0.441439,0.311748,0.205877,-0.197306,0.308774,1,+7817+3248,5,0.000364,"{'geodesic': False, 'type': 'Polygon', 'coordi...",Iowa,Plymouth,2022-05-08
2,1.408239,0.572035,0.590689,0.068866,0.885908,3,+7752+3323,121,0.000855,"{'geodesic': False, 'type': 'MultiPolygon', 'c...",Iowa,Plymouth,2022-05-08
3,0.920479,0.481340,0.418542,-0.083757,0.627733,1,+7768+3250,176,0.001210,"{'geodesic': False, 'type': 'Polygon', 'coordi...",Iowa,Plymouth,2022-05-08
4,0.345693,0.313350,0.185989,-0.214592,0.278950,8,+7773+3270,1,0.001371,"{'geodesic': False, 'type': 'Polygon', 'coordi...",Iowa,Plymouth,2022-05-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0.681630,0.360597,0.267591,-0.130399,0.401343,1,+7962+3942,176,0.066350,"{'geodesic': False, 'type': 'Polygon', 'coordi...",Iowa,Mills,2022-05-08
396,0.538998,0.296132,0.202911,-0.160893,0.304323,4,+7974+3941,111,0.067303,"{'geodesic': False, 'type': 'Polygon', 'coordi...",Iowa,Mills,2022-05-08
397,0.593875,0.338817,0.238203,-0.188054,0.357264,1,+8023+3942,121,0.067613,"{'geodesic': False, 'type': 'Polygon', 'coordi...",Iowa,Mills,2022-05-08
398,2.209502,0.579548,0.647332,0.179870,0.970879,1,+7918+3866,141,0.067669,"{'geodesic': False, 'type': 'Polygon', 'coordi...",Iowa,Mills,2022-05-08


In [37]:
all_data['corn'] = all_data['label'].apply(lambda x: 1 if x == 1 else 0)

In [42]:
# all_data.drop(['geometry','random','count'], axis=1, inplace=True)

In [48]:
all_data

,EVI,GNDVI,NDVI,NDWI,SAVI,field_id,state,county,crop
0,0.628393,0.404195,0.298536,-0.163510,0.447747,+7735+3224,Iowa,Plymouth,0
1,0.441439,0.311748,0.205877,-0.197306,0.308774,+7817+3248,Iowa,Plymouth,0
2,1.408239,0.572035,0.590689,0.068866,0.885908,+7752+3323,Iowa,Plymouth,0
3,0.920479,0.481340,0.418542,-0.083757,0.627733,+7768+3250,Iowa,Plymouth,0
4,0.345693,0.313350,0.185989,-0.214592,0.278950,+7773+3270,Iowa,Plymouth,1
...,...,...,...,...,...,...,...,...,...
395,0.681630,0.360597,0.267591,-0.130399,0.401343,+7962+3942,Iowa,Mills,0
396,0.538998,0.296132,0.202911,-0.160893,0.304323,+7974+3941,Iowa,Mills,0
397,0.593875,0.338817,0.238203,-0.188054,0.357264,+8023+3942,Iowa,Mills,0
398,2.209502,0.579548,0.647332,0.179870,0.970879,+7918+3866,Iowa,Mills,0


In [44]:
def train_test_val(df, y, test=0.2, val=0.2, random_state=1):
    df_full_train, df_test = train_test_split(df, test_size=test, random_state=random_state)
    df_train, df_val = train_test_split(df_full_train, test_size= val/(1-test) , random_state=random_state)
    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop=True)
    df_test = df_test.reset_index(drop=True)
    y_train = df_train[y].values
    y_val = df_val[y].values
    y_test = df_test[y].values
    del df_train[y]
    del df_val[y]
    del df_test[y]
    return df_train, df_val, df_test, y_train, y_val, y_test

In [50]:
df_train, df_val, df_test, y_train, y_val, y_test = train_test_val(all_data, 'crop')

In [51]:
# One-hot encoding of variables
dv = DictVectorizer(sparse=False)

train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

# Training logistic regression
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=1)
model.fit(X_train, y_train)

# Calculating accuracy
y_pred = model.predict_proba(X_val)[:, 1]
corn = (y_pred) >= 0.5

In [54]:
corn

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False])